In [2]:
#Download the data
import yfinance as yf

symbols = ['VTI', 'AGG', 'DBC', '^VIX']  # Google (GOOG) and Meta Platforms Inc. (META) tickers
# period = '1y'  # 1 month of historical data (you can adjust the period as per your requirement)

# data = yf.download(symbols, period=period)['Close']
data = yf.download(symbols, start='2021-01-01', end='2021-12-30')['Close']
print(data)

[*********************100%***********************]  4 of 4 completed
                   AGG        DBC         VTI       ^VIX
Date                                                    
2021-01-04  118.040001  14.620000  191.869995  26.969999
2021-01-05  117.919998  15.030000  193.419998  25.340000
2021-01-06  117.339996  15.050000  195.160004  25.070000
2021-01-07  117.220001  15.120000  198.289993  22.370001
2021-01-08  117.080002  15.240000  199.250000  21.559999
...                ...        ...         ...        ...
2021-12-22  114.339996  20.549999  239.020004  18.629999
2021-12-23  114.199997  20.660000  240.669998  17.959999
2021-12-27  114.260002  20.920000  242.960007  17.680000
2021-12-28  114.209999  20.879999  242.460007  17.540001
2021-12-29  113.870003  20.980000  242.600006  16.950001

[250 rows x 4 columns]


In [3]:
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping

class Model:
    def __init__(self, input_shape, n_outputs, data=data):
        self.data = data # remove
        self.model = self.build_model(input_shape, n_outputs)
        self.callback = EarlyStopping(monitor='loss', min_delta=0.001, patience=4)
    def build_model(self, input_shape, outputs):
        model = Sequential([
            LSTM(64, input_shape=input_shape, activation='relu'),
            Flatten(),
            Dense(outputs, activation='softmax')
        ])

        def sharpe_loss(_, y_pred):
            coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
            portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

            sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            print(type(sharpe))
            
            # exp keeps relative ordering between positives and negatives
            #   since we want to maximize sharp, while gradient descent minimizes the loss
            #   we negate the Sharpe value
            return K.exp(-sharpe)
        
        model.compile(loss=sharpe_loss, optimizer='adam')
        return model

    def fit(self, data, labels):
        self.model.fit(data, labels, epochs=20, shuffle=False) #, callbacks=[self.callback])
        
    def get_allocations(self, data):
        return self.model.predict_proba(data)

t = tf.constant(data)
t = tf.cast(t, tf.float32)

model = Model((t.shape[0], t.shape[1]), t.shape[1], t)

2023-08-11 21:17:33.910653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 21:17:33.912623: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) Graphics)
2023-08-11 21:17:34.022164: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 21:17:34.022236: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2023-08-11 21:17:34.022289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

In [7]:
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
import pandas as pd

class Model:
    def __init__(self):
        self.data = None
        self.model = None
        
        # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
    def __build_model(self, input_shape, outputs):
        model = Sequential([
            LSTM(64, input_shape=input_shape, activation='relu'),
            Flatten(),
            Dense(outputs, activation='softmax')
        ])

        def sharpe_loss(_, y_pred):
            coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
            portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
            portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

            sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
            # exp keeps relative ordering between positives and negatives
            #   since we want to maximize sharp, while gradient descent minimizes the loss
            #   we negate the Sharpe value
            return K.exp(-sharpe)
        
        model.compile(loss=sharpe_loss, optimizer='adam')
        return model
    
    def get_allocations(self, data):
        
        
        # data with returns
        data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
        data = data.iloc[1:]
        self.data = tf.cast(tf.constant(data), float)
        
        if self.model is None:
            self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
        fit_predict_data = data_w_ret[np.newaxis,:]        
        self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
        return self.model.predict_proba(fit_predict_data)[0]

In [5]:
input_data = data.values[np.newaxis, :]
y = np.array([[i for i in range(t.shape[1])]]).astype(float)

model.fit(input_data, y)

Epoch 1/20
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


2023-08-11 21:17:45.339768: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-11 21:17:45.673529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 21:17:45.673646: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7825 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-08-11 21:17:45.674194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 21:17:45.674227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_fa

1/1 [==============================] - 4s 4s/step - loss: 0.8915
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 7/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 8/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 9/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 10/20
1/1 [==============================] - 2s 2s/step - loss: 0.9759
Epoch 11/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 12/20
1/1 [==============================] - 2s 2s/step - loss: 0.8915
Epoch 13/20
1/1 [==============================] - 3s 3s/step - loss: 0.8915
Epoch 14/20
1/1 [==

In [8]:
model = Model()
x = model.get_allocations(pd.DataFrame(np.random.rand(5, 5)))

class_probabilities = model.model.predict(x[np.newaxis, :])[0]

Epoch 1/10


2023-08-11 21:20:39.394704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.9240
Epoch 2/10
1/1 [==============================] - 0s 38ms/step - loss: 0.9094
Epoch 3/10
1/1 [==============================] - 0s 48ms/step - loss: 0.8962
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 0.8840
Epoch 5/10


2023-08-11 21:20:39.717357: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 21:20:39.717432: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7825 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-08-11 21:20:39.717994: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-11 21:20:39.718025: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7825 MB memory) -> physical PluggableDevice (device: 0, name: DM

1/1 [==============================] - 0s 31ms/step - loss: 0.8730
Epoch 6/10
1/1 [==============================] - 0s 58ms/step - loss: 0.8630
Epoch 7/10
1/1 [==============================] - 0s 33ms/step - loss: 0.8536
Epoch 8/10
1/1 [==============================] - 0s 34ms/step - loss: 0.8447
Epoch 9/10
1/1 [==============================] - 0s 32ms/step - loss: 0.8362
Epoch 10/10
1/1 [==============================] - 0s 30ms/step - loss: 0.8281


AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [9]:
pd.DataFrame(np.random.rand(5,5))

,0,1,2,3,4
0,0.921521,0.884176,0.846435,0.033654,0.856059
1,0.605879,0.896506,0.559413,0.906368,0.117317
2,0.937482,0.285782,0.822959,0.699012,0.039574
3,0.366828,0.343420,0.197947,0.295201,0.908305
4,0.480537,0.069265,0.944748,0.543391,0.936808


In [52]:
len(data.columns)

4

In [53]:
[x.split(' ')[0] for x in data.columns]

['AGG', 'DBC', 'VTI', '^VIX']

In [54]:
import tensorflow as tf
tf.constant([1,2,3])[0]

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [55]:
import tensorflow as tf
tf.constant(data)

<tf.Tensor: shape=(250, 4), dtype=float64, numpy=
array([[118.04000092,  14.61999989, 191.86999512,  26.96999931],
       [117.91999817,  15.02999973, 193.41999817,  25.34000015],
       [117.33999634,  15.05000019, 195.16000366,  25.06999969],
       [117.22000122,  15.11999989, 198.28999329,  22.37000084],
       [117.08000183,  15.23999977, 199.25      ,  21.55999947],
       [116.88999939,  15.13000011, 198.05999756,  24.07999992],
       [116.98999786,  15.42000008, 198.75999451,  23.32999992],
       [117.36000061,  15.38000011, 198.97999573,  22.20999908],
       [117.12999725,  15.52000046, 198.78999329,  23.25      ],
       [117.25      ,  15.30000019, 197.05999756,  24.34000015],
       [117.41000366,  15.35000038, 198.86000061,  23.23999977],
       [117.41999817,  15.35999966, 201.36999512,  21.57999992],
       [117.25      ,  15.35000038, 201.22000122,  21.31999969],
       [117.25      ,  15.09000015, 200.88999939,  21.90999985],
       [117.55000305,  15.25      , 201.